## Part 1: Explore the Data

Import the data and use Pandas to learn more about the dataset.

In [62]:
import pandas as pd

df = pd.read_csv('Resources/client_dataset.csv')

df.head()

,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,item_id,category,subcategory,unit_price,unit_cost,unit_weight,qty,line_number
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,EUD29711-63-6U,decor,wall art,1096.80,762.71,7.50,105,1
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,XDA18116-89-4A,consumables,pens,24.95,15.09,1.49,21,0
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,ABE59463-05-7E,software,project management,13.52,7.86,1.68,39,6
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,ZMM00836-65-0C,consumables,pens,36.42,24.85,1.23,29,3
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,BZX55559-12-3X,consumables,misc,195.10,108.17,46.43,20,1


In [63]:
# View the column names in the data
df.columns


Index(['first', 'last', 'job', 'phone', 'email', 'client_id', 'order_id',
       'order_date', 'order_week', 'order_year', 'item_id', 'category',
       'subcategory', 'unit_price', 'unit_cost', 'unit_weight', 'qty',
       'line_number'],
      dtype='object')

In [64]:
# Use the describe function to gather some basic statistics
df.describe()


,client_id,order_id,order_week,order_year,unit_price,unit_cost,unit_weight,qty,line_number
count,54639.000000,5.463900e+04,54639.000000,54639.000000,54639.000000,54639.000000,54639.000000,5.463900e+04,54639.000000
mean,54837.869416,5.470190e+06,11.359139,2022.993064,136.267207,99.446073,5.004116,5.702646e+02,2.979667
std,25487.438231,2.599807e+06,7.023499,0.082997,183.873135,133.164267,5.326599,1.879552e+04,2.436320
min,10033.000000,1.000886e+06,1.000000,2022.000000,0.010000,0.010000,0.000000,0.000000e+00,0.000000
25%,33593.000000,3.196372e+06,6.000000,2023.000000,20.800000,14.840000,1.450000,3.200000e+01,1.000000
50%,53305.000000,5.496966e+06,11.000000,2023.000000,68.310000,49.890000,3.240000,6.800000e+01,3.000000
75%,78498.000000,7.733869e+06,17.000000,2023.000000,173.160000,125.570000,6.890000,1.700000e+02,5.000000
max,99984.000000,9.998480e+06,52.000000,2023.000000,1396.230000,846.270000,46.430000,3.958244e+06,9.000000


In [65]:
# Use this space to do any additional research
# and familiarize yourself with the data.
df['category'].value_counts()


category
consumables    23538
furniture      11915
software        8400
decor           7954
electronics     2832
Name: count, dtype: int64

In [66]:
# What three item categories had the most entries?

# consumables    23538
# furniture      11915
# software        8400
# Name: category, dtype: int64

category_counts = df['category'].value_counts()
category_counts = pd.DataFrame(category_counts)
category_counts = category_counts.sort_values('count', ascending=False)
category_counts.head(3)


,count
category,
consumables,23538
furniture,11915
software,8400


In [106]:
# For the category with the most entries,
# which subcategory had the most entries?

# bathroom supplies    6424
# Name: subcategory, dtype: int64

subcategories = df.loc[
    df['category'] == 'consumables',
    ['category', 'subcategory']
]
subcats = pd.DataFrame(subcategories.value_counts())
subcats = subcats.reset_index().sort_values('count', ascending=False)
subcats.head(1)


,category,subcategory,count
0,consumables,bathroom supplies,6424


In [68]:
# Which five clients had the most entries in the data?

# 33615    220
# 66037    211
# 46820    209
# 38378    207
# 24741    207
# Name: client_id, dtype: int64

client_entry_counts = pd.DataFrame(df['client_id'].value_counts())
client_entry_counts = client_entry_counts.sort_values('count', ascending=False)
client_entry_counts.head()

,count
client_id,
33615,220
66037,211
46820,209
38378,207
24741,207


In [69]:
# Store the client ids of those top 5 clients in a list.
# [33615, 66037, 46820, 38378, 24741]
# top_5_clients = pd.DataFrame(df['client_id'].value_counts().head())
top_5_clients_list = list(client_entry_counts.head().index)
top_5_clients_list


[33615, 66037, 46820, 38378, 24741]

In [70]:
# How many total units (the qty column) did the
# client with the most entries order order?
# 64313
top_client_qtys = df.loc[df['client_id'] == 33615, ['client_id', 'qty']]
total_qty = sum(list(top_client_qtys['qty']))
total_qty


64313

## Part 2: Transform the Data
Do we know that this client spent the more money than client 66037? If not, how would we find out? Transform the data using the steps below to prepare it for analysis.

In [71]:
# Create a column that calculates the 
# subtotal for each line using the unit_price
# and the qty

# unit_price	qty	line_subtotal
# 0	1096.80	105	115164.00
# 1	24.95	21	523.95

df_w_subtotal = df.copy()
df_w_subtotal['line_subtotal'] = df.unit_price * df.qty
df_w_subtotal[['unit_price','qty','line_subtotal']].head()

,unit_price,qty,line_subtotal
0,1096.80,105,115164.00
1,24.95,21,523.95
2,13.52,39,527.28
3,36.42,29,1056.18
4,195.10,20,3902.00


In [72]:
# Create a column for shipping price.
# Assume a shipping price of $7 per pound
# for orders over 50 pounds and $10 per
# pound for items 50 pounds or under.

# unit_price	unit_weight	qty	total_weight	shipping_price
# 0	1096.80	        7.50	105	    787.50	    5512.50
# 1	24.95	        1.49	21	    31.29	    312.90
# 2	13.52	        1.68	39	    65.52	    458.64

df_w_shipping = df_w_subtotal.copy()
df_w_shipping['total_weight'] = df.unit_weight * df.qty
df_w_shipping['shipping_price'] = df_w_shipping.total_weight.apply(
    lambda x: 7 * x if x > 50 else 10 * x
)
df_w_shipping[
    ['unit_price', 'unit_weight', 'qty', 'total_weight', 'shipping_price']
].head()


,unit_price,unit_weight,qty,total_weight,shipping_price
0,1096.80,7.50,105,787.50,5512.50
1,24.95,1.49,21,31.29,312.90
2,13.52,1.68,39,65.52,458.64
3,36.42,1.23,29,35.67,356.70
4,195.10,46.43,20,928.60,6500.20


In [73]:
# Create a column for the total price
# using the subtotal and the shipping price
# along with a sales tax of 9.25%

# line_subtotal	shipping_price	line_price
# 0	115164.00	5512.50	131839.08
# 1	523.95	312.90	914.26
# 2	527.28	458.64	1077.12

df_w_tax = df_w_shipping.copy()
df_w_tax['line_price'] = round(
    1.0925 * (df_w_tax.line_subtotal + df_w_tax.shipping_price),
    2
)

df_w_tax[['line_subtotal', 'shipping_price', 'line_price']].head()

,line_subtotal,shipping_price,line_price
0,115164.00,5512.50,131839.08
1,523.95,312.90,914.26
2,527.28,458.64,1077.12
3,1056.18,356.70,1543.57
4,3902.00,6500.20,11364.40


In [74]:
# Create a column for the cost
# of each line using unit cost, qty, and
# shipping price (assume the shipping cost
# is exactly what is charged to the client).

# first	last	job	phone	email	client_id	order_id	order_date	order_week	order_year	...	unit_price	unit_cost	unit_weight	qty	line_number	line_subtotal	total_weight	shipping_price	line_price	line_cost
# 0	Donald	Harding	Immunologist	793-904-7725x39308	harding.donald.7185@sullivan.com	58515	8953482	2023-04-28	17	2023	...	1096.80	762.71	7.50	105	1	115164.00	787.50	5512.50	131839.08	85597.05
# 1	Tiffany	Myers	Music therapist	201.442.4543x942	myers.t.6537@ferguson-johnson.net	37609	8069089	2023-05-19	20	2023	...	24.95	15.09	1.49	21	0	523.95	31.29	312.90	914.26	629.79
# 2	Shannon	Watson	Immunologist	687.737.9424x8503	swatson8146@payne.net	57113	1902144	2023-01-29	4	2023	...	13.52	7.86	1.68	39	6	527.28	65.52	458.64	1077.12	765.18

df_w_cost = df_w_tax.copy()
df_w_cost['line_cost'] = df_w_cost.apply(
    lambda r: r.unit_cost * r.qty + r.shipping_price,
    axis=1
)
df_w_cost.head()


,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,...,unit_price,unit_cost,unit_weight,qty,line_number,line_subtotal,total_weight,shipping_price,line_price,line_cost
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,...,1096.80,762.71,7.50,105,1,115164.00,787.50,5512.50,131839.08,85597.05
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,...,24.95,15.09,1.49,21,0,523.95,31.29,312.90,914.26,629.79
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,...,13.52,7.86,1.68,39,6,527.28,65.52,458.64,1077.12,765.18
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,...,36.42,24.85,1.23,29,3,1056.18,35.67,356.70,1543.57,1077.35
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,...,195.10,108.17,46.43,20,1,3902.00,928.60,6500.20,11364.40,8663.60


In [75]:
# Create a column for the profit of
# each line using line cost and line price

# 	first	last	job	phone	email	client_id	order_id	order_date	order_week	order_year	...	unit_cost	unit_weight	qty	line_number	line_subtotal	total_weight	shipping_price	line_price	line_cost	line_profit
# 0	Donald	Harding	Immunologist	793-904-7725x39308	harding.donald.7185@sullivan.com	58515	8953482	2023-04-28	17	2023	...	762.71	7.50	105	1	115164.00	787.50	5512.50	131839.08	85597.05	46242.03
# 1	Tiffany	Myers	Music therapist	201.442.4543x942	myers.t.6537@ferguson-johnson.net	37609	8069089	2023-05-19	20	2023	...	15.09	1.49	21	0	523.95	31.29	312.90	914.26	629.79	284.47
# 2	Shannon	Watson	Immunologist	687.737.9424x8503	swatson8146@payne.net	57113	1902144	2023-01-29	4	2023	...	7.86	1.68	39	6	527.28	65.52	458.64	1077.12	765.18	311.94

df_w_profit = df_w_cost.copy()
df_w_profit['line_profit'] = df_w_profit['line_price']\
    - df_w_profit['line_cost']
df_w_profit.head()


,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,...,unit_cost,unit_weight,qty,line_number,line_subtotal,total_weight,shipping_price,line_price,line_cost,line_profit
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,...,762.71,7.50,105,1,115164.00,787.50,5512.50,131839.08,85597.05,46242.03
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,...,15.09,1.49,21,0,523.95,31.29,312.90,914.26,629.79,284.47
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,...,7.86,1.68,39,6,527.28,65.52,458.64,1077.12,765.18,311.94
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,...,24.85,1.23,29,3,1056.18,35.67,356.70,1543.57,1077.35,466.22
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,...,108.17,46.43,20,1,3902.00,928.60,6500.20,11364.40,8663.60,2700.80


## Part 3: Confirm your work
You have email receipts showing that the total prices for 3 orders. Confirm that your calculations match the receipts. Remember, each order has multiple lines.

Order ID 2742071 had a total price of \$152,811.89

Order ID 2173913 had a total price of \$162,388.71

Order ID 6128929 had a total price of \$923,441.25


In [76]:
# Check your work using the totals above

# Order 2742071 total: $152811.89
# Order 2173913 total: $162388.71
# Order 6128929 total: $923441.25

for order in [2742071, 2173913, 6128929]:
    total = round(
        df_w_profit.loc[df_w_profit['order_id'] == order, 'line_price'].sum(), 2
    )
    print(f"Order {order} total: ${total:,}")


Order 2742071 total: $152,811.89
Order 2173913 total: $162,388.71
Order 6128929 total: $923,441.25


## Part 4: Summarize and Analyze
Use the new columns with confirmed values to find the following information.

In [77]:
# How much did each of the top 5 clients by quantity
# spend? Check your work from Part 1 for client ids.

# 33615: $8377308.52
# 66037: $10259514.79
# 46820: $9743794.36
# 38378: $12906550.87
# 24741: $82268892.02

top_5_clients_list
client_totals = df_w_profit.copy()[[
    'client_id',
    'qty',
    'line_subtotal',
    'total_weight',
    'shipping_price',
    'line_price',
    'line_cost',
    'line_profit'
]]
client_totals = client_totals.groupby(['client_id']).sum().reset_index()
top_5_spend = client_totals.loc[
    client_totals['client_id'].isin(top_5_clients_list),
    ['client_id', 'line_price']
]

top_5_spend.line_price = top_5_spend.line_price.apply(lambda x: f"${x:,.2f}")
top_5_spend


,client_id,line_price
144,24741,"$82,268,892.02"
258,33615,"$8,377,308.52"
319,38378,"$12,906,550.87"
402,46820,"$9,743,794.36"
621,66037,"$10,259,514.79"


In [82]:
# Create a summary DataFrame showing the totals for the
# for the top 5 clients with the following information:
# total units purchased, total shipping price,
# total revenue, and total profit. Sort by total profit.

# client_id	qty	shipping_price	line_price	line_cost	line_profit
# 4	24741	239862	5126448.37	82268892.02	45688899.71	36579992.31
# 3	38378	73667	3429455.40	12906550.87	9634720.98	3271829.89
# 1	66037	43018	1395151.85	10259514.79	7004482.98	3255031.81
# 2	46820	75768	1601448.84	9743794.36	7007191.64	2736602.72
# 0	33615	64313	1828984.89	8377308.52	6175313.91	2201994.61

top_5_totals = client_totals\
    .loc[
        client_totals['client_id'].isin(top_5_clients_list),
        [
            'client_id',
            'qty',
            'shipping_price',
            'line_price',
            'line_cost',
            'line_profit'
        ]
    ]\
    .sort_values('line_profit', ascending=False)\
    .head()
top_5_totals

,client_id,qty,shipping_price,line_price,line_cost,line_profit
144,24741,239862,5126448.37,82268892.02,45688899.71,36579992.31
319,38378,73667,3429455.40,12906550.87,9634720.98,3271829.89
621,66037,43018,1395151.85,10259514.79,7004482.98,3255031.81
402,46820,75768,1601448.84,9743794.36,7007191.64,2736602.72
258,33615,64313,1828984.89,8377308.52,6175313.91,2201994.61


In [105]:
# Format the data and rename the columns
# to names suitable for presentation.
# Currency should be in millions of dollars.

def format_and_scale(row):
    # Remove ".0" from qty and present as whole numbers
    row.qty = f"{row.qty:.0f}"
    # Scale and add units to monetary values
    for col in ['shipping_price', 'line_price', 'line_cost', 'line_profit']:
        row[col] = f"${round(row[col]/1_000_000, 2):.2f}M"
    return row

top_5_formatted = top_5_totals.copy()
top_5_formatted = top_5_formatted.apply(format_and_scale, axis=1)
top_5_formatted = top_5_formatted.rename(columns={
    'client_id': 'Client ID',
    'qty': 'Units',
    'shipping_price': 'Shipping',
    'line_price': 'Total Revenue',
    'line_cost': 'Total Cost',
    'line_profit': 'Total Profit'
})

In [104]:
# Sort the updated data by "Total Profit" form highest to lowest

# Client ID	Units	Shipping	Total Revenue	Total Cost	Total Profit
# 4	24741	239862	$5.13M	$82.27M	$45.69M	$36.58M
# 3	38378	73667	$3.43M	$12.91M	$9.63M	$3.27M
# 1	66037	43018	$1.40M	$10.26M	$7.00M	$3.26M
# 2	46820	75768	$1.60M	$9.74M	$7.01M	$2.74M
# 0	33615	64313	$1.83M	$8.38M	$6.18M	$2.20M

# top_5_totals was already sorted, but sorting again just to make sure
top_5_formatted.sort_values('Total Profit', ascending=False)

,Client ID,Units,Shipping,Total Revenue,Total Cost,Total Profit
144,24741.0,239862,$5.13M,$82.27M,$45.69M,$36.58M
319,38378.0,73667,$3.43M,$12.91M,$9.63M,$3.27M
621,66037.0,43018,$1.40M,$10.26M,$7.00M,$3.26M
402,46820.0,75768,$1.60M,$9.74M,$7.01M,$2.74M
258,33615.0,64313,$1.83M,$8.38M,$6.18M,$2.20M
